In [6]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 50)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from google.cloud import bigquery
from google.oauth2 import service_account
from pathlib import Path
from sklearn import set_config; set_config(display='diagram')
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer
from flaml import AutoML



In [4]:
df = pd.read_csv('../../data/clean_data_final.csv')
df = df[df['PFPI_MINUTES']!=0]

In [7]:
num_transformer = MinMaxScaler()

cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False)

transformer = make_column_transformer((num_transformer, ['Lat_OR','Lon_OR', 'Lat_DES','Lon_DES']),
                                  (cat_transformer, ['ENGLISH_DAY_TYPE', 'SERVICE_GROUP_CODE_AFFECTED', 'INCIDENT_REASON',
                                                        'UNIT_CLASS_AFFECTED', 'TRAIN_SERVICE_CODE_AFFECTED', 
                                                     'PERFORMANCE_EVENT_CODE', 
                                                     'APP_TIMETABLE_FLAG_AFF']),
                                remainder = 'passthrough')


pipe = Pipeline([('transformer', transformer)])

pipe

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  ['Lat_OR', 'Lon_OR',
                                                   'Lat_DES', 'Lon_DES']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['ENGLISH_DAY_TYPE',
                                                   'SERVICE_GROUP_CODE_AFFECTED',
                                                   'INCIDENT_REASON',
                                                   'UNIT_CLASS_AFFECTED',
                                                   'TRAIN_SERVICE_CODE_AFFECTED',
                                                   'PERFORMANCE_EVENT_CODE',
                                                   'APP_TIMETABLE_FLAG_AFF'])]))])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='PFPI_MINUTES'), df['PFPI_MINUTES'], test_size=0.3)

In [9]:
X_train_scaled = pipe.fit_transform(X_train)
X_test_scaled = pipe.transform(X_test)


In [11]:
import pickle

model = pickle.load(open('../../../BestTrainDelays/streamlit/Models/XGBoost_RMSE_on_test_with_log_transf_rmse_1_64.sav', 'rb'))

/home/leo/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.2.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/leo/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ColumnTransformer from version 1.2.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
model

AutoML(append_log=False, auto_augment=True, custom_hp={},
       cv_score_agg_func=None, early_stop=False, ensemble=False,
       estimator_list='auto', eval_method='auto', fit_kwargs_by_estimator={},
       force_cancel=False, free_mem_ratio=0, hpo_method='auto',
       keep_search_state=False, learner_selector='sample', log_file_name='',
       log_training_metric=False, log_type='better', max_iter=None,
       mem_thres=4294967296, metric='auto', metric_constraints=[],
       min_sample_size=10000, mlflow_logging=True, model_history=False,
       n_concurrent_trials=1, n_jobs=-1, n_splits=5, pred_time_limit=inf,
       preserve_checkpoint=True, retrain_full=True, ...)

In [20]:
model.predict(X_train_scaled).argmax()

25974

In [23]:
X_train_scaled[25974]

array([ 0.78720523,  0.55947023,  0.78720523,  0.55947023,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  1.        ,  0.58778525,  0.80901699,
        0.91354546,  0.40673664,  0.10452846,  0.9945219 , -0.8660254 ,
       -0.5       ,  0.58778525,  0.80901699,  0.91354546,  0.40673664,
        0.10452846,  0.9945219 , -0.80901699,  0.58778525])

In [27]:
X_train.iloc[25974,:]

TRAIN_SERVICE_CODE_AFFECTED    21252001
SERVICE_GROUP_CODE_AFFECTED        EK99
ENGLISH_DAY_TYPE                     WD
APP_TIMETABLE_FLAG_AFF                Y
UNIT_CLASS_AFFECTED               315.0
INCIDENT_REASON                       O
PERFORMANCE_EVENT_CODE                M
Lat_OR                         51.63309
Lon_OR                          0.00989
Lat_DES                        51.63309
Lon_DES                         0.00989
ORIG_MONTH_SIN                 0.587785
ORIG_MONTH_COS                 0.809017
ORIG_DAY_SIN                   0.913545
ORIG_DAY_COS                   0.406737
ORIG_HOUR_SIN                  0.104528
ORIG_HOUR_COS                  0.994522
ORIG_MINUTE_SIN               -0.866025
ORIG_MINUTE_COS                    -0.5
DEST_MONTH_SIN                 0.587785
DEST_MONTH_COS                 0.809017
DEST_DAY_SIN                   0.913545
DEST_DAY_COS                   0.406737
DEST_HOUR_SIN                  0.104528
DEST_HOUR_COS                  0.994522
